<a href="https://colab.research.google.com/github/TheRoberto2512/DeepBrainMRI/blob/main/Pre%20Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size=6>**FACE MORPHING DETECTION: PRE-PROCESSING**</font>
</br><font size=3>*Roberto A. Usai, Davide Senette, Chiara Scalas*</font>



<p align="justify">In questo notebook si può gestire il ridimensionamento delle immagini e il bilanciamento del dataset adottando un approccio misto tra data augmentation e random undersampling. Le funzioni sono parametrizzate, pertanto è possibile scegliere quante immagini generare con la data augmentation e quante eliminarle tramite random undersampling.
<br><br>
Infine, è possibile comprimere il dataset appena bilanciato e scaricarlo in formato .zip, in modo da poterlo utilizzare facilmente con gli altri notebook del progetto.</p>

**Indice:**
*   [Import librerie e impostazioni](#1)
*   [Download del Dataset](#2)
*   [Resize delle immagini](#3)
*   [Bilanciamento](#4)
*   [Salvataggio](#5)

<a name="1"></a>
# **Import librerie e impostazioni**

In [ ]:
!pip install python-resize-image # libreria per fare il resize delle immagini

In [ ]:
from resizeimage import resizeimage
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
from google.colab import drive, files
import imageio.v2 as imageio
from PIL import Image
import numpy as np
import zipfile
import random
import gdown
import json
import sys
import os

Prima importiamo le librerie, poi montiamo Google Drive per poter accedere facilmente agli altri file.

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Path della cartella del progetto su Google Drive:

#@markdown <font color="#ed7d31"><b>Necessario per poter accedere agli altri file</b>!</font>
#@markdown <br>Se la cartella del progetto si trova nella root di Drive, scrivere solo il suo nome:
DRIVE_PATH = "MAD Project" #@param {type:"string"}

DRIVE_PATH = '/content/drive/MyDrive/' + DRIVE_PATH

WEIGHTS_PATH = DRIVE_PATH + '/Weights/'

In [ ]:
sys.path.append(DRIVE_PATH)                       # ci permetterà di importare le funzioni presenti in altri file
from shared_utilities import download_dataset, move_files, split_dataset

Scarichiamo dal file .json gli ID necessari per il download del dataset e del csv.

In [ ]:
with open(DRIVE_PATH + '/settings.json', 'r') as file:
  config = json.load(file)

DATASET_ID = config['DATASET_ID']

In [ ]:
#@title Impostazioni del Notebook

#@markdown Seme per le funzioni randomiche del notebook:
SEED = 2407 #@param {type:"integer"}

random.seed(SEED)

np.set_printoptions(suppress=True) # NumPy non utilizzerà la notazione scientifica per piccoli numeri, rendendo l'output più leggibile.

<a name="2"></a>
# **Download del dataset**


In [ ]:
#@title Impostazioni download

#@markdown Nome del zip dataset post download:
DATASET_NAME = 'AMSL_dataset.zip' #@param {type:"string"}

download_dataset(DATASET_ID, DATASET_NAME, msg=True)

Dopo aver scaricato il dataset lo manipoliamo in modo da unzipparlo, creare le directory per gli split e infine eliminare i file txt e la cartella sample_data creata automaticamente da Colab.

In [ ]:
%%capture
# evita il fastidioso output a video

! unzip "{DATASET_NAME}"                                  # unzippa il file zip
! rm /content/AMSL/*.txt                                  # elimina i file txt
! rm -r /content/sample_data                              # elimina la cartella di default di Colab
! mv /content/AMSL/neutral/* /content/AMSL/smiling        # sposta i file della cartella neutral in smiling
! mv /content/AMSL/smiling /content/AMSL/bona_fide        # rinomina smiling in bona_fide
! rmdir AMSL/neutral                                      # rimuove la cartella neutral (ormai vuota)

<a name="3"></a>
# **Resize delle immagini**


In [ ]:
#@title Dimensioni per il resize delle immagini:

#@markdown Larghezza dell'immagine:
IMAGE_WIDTH = 224 #@param {type: "integer"}

#@markdown Altezza dell'immagine:
IMAGE_HEIGHT = 224 #@param {type: "integer"}

dict_people = {} # salverà il numero di immagini per persona

# -- -- #  -- -- # -- -- # -- -- # -- -- # -- -- # -- -- #

def resize_images(image_folder, output_folder, target_size):
  '''
  Funzione per fare la resize delle immagini.

  Parametri:
  - image_folder: cartella con le immagini da ridimensionare.
  - output_folder: cartella dove salvare le immagini ridimensionate.
  - target_size: tupla (width, height).
  '''

  if not os.path.exists(output_folder):                                           # crea la directory per le immagini ridimensionate se non esiste
    os.makedirs(output_folder)

  for image_name in os.listdir(image_folder):                                     # itera attraverso tutte le directory (classi) nell'input folder
    class_input_folder = os.path.join(image_folder, image_name)                   # path della cartella della classe specifica in input
    class_output_folder = os.path.join(output_folder, image_name)                 # path della cartella della classe specifica nell'output folder

    if not os.path.exists(class_output_folder):                                   # crea la directory della classe per le immagini ridimensionate se non esiste
      os.makedirs(class_output_folder)

    for image_name in os.listdir(class_input_folder):                             # itera attraverso ogni immagine nella cartella della classe
      image_path = os.path.join(class_input_folder, image_name)                   # path completo dell'immagine
      if os.path.isfile(image_path):                                              # verifica se è un file
        with open(image_path, 'r+b') as f:                                        # apre l'immagine
          with Image.open(f) as image:
            cover = resizeimage.resize_cover(image, target_size)                  # ridimensiona l'immagine alla dimensione target
            output_image_path = os.path.join(class_output_folder, image_name)     # path completo per salvare l'immagine ridimensionata
            #print(f"Saving resized image to: {output_image_path}")               # debug: stampa il path dove sarà salvata l'immagine ridimensionata
            cover.save(output_image_path, image.format)                           # salva l'immagine ridimensionata nel path specificato

# -- -- #  -- -- # -- -- # -- -- # -- -- # -- -- # -- -- #

resize_images('/content/AMSL', '/content/Resized', target_size = (IMAGE_WIDTH, IMAGE_HEIGHT))

<a name="4"></a>
# **Bilanciamento**


Costruiamo il modello che si occuperà di fare la data augmentation delle immagini.

In [ ]:
imageGenerator = iaa.Sequential([
    iaa.Fliplr(0.5),                # flip orizzontale con probabilità del 50%
    iaa.Multiply((0.8, 1.2)),       # modificare la luminosità
    iaa.Affine(scale=(1, 1.2))      # zoom in avanti fino al 20%
])

In [ ]:
#@title Parametri per il bilanciamento

#@markdown Numero di immagini da generare per ogni immagine bona_fide:
multiplier = 6 #@param {type:"integer"}

#@markdown Numero massimo di immagini da morph:
max_morph_samples = 1326 #@param {type:"integer"}

# -- -- # -- -- # -- -- # -- -- # -- -- # -- -- # -- -- #

def dataAugmentation(imgPath, multiplier=6):
  '''
  Funzione che prende in input il percorso di un'immagine e restituisce le immagini generate a partire da lei.

  Parametri:
  - imgPath: percorso dell'immagine da trasformare.
  - multiplier: numero di immagini generate per ogni immagine originale.

  Restituisce:
  - augmented_images: lista di immagini generate.
  '''

  image = imageio.imread(imgPath)                                                     # legge l'immagine
  image_array = np.array(image)                                                       # converte l'immagine in un array numpy

  augmented_images = imageGenerator(images=[image_array for _ in range(multiplier)])  # genera immagini tramite l'imageGenerator

  return augmented_images

# -- -- # -- -- # -- -- # -- -- # -- -- # -- -- # -- -- #

def saveToFile(imgs, image_name, folder):
  '''
  Funzione per salvare le immagini in un file.

  Parametri:
  - imgs: lista di immagini da salvare.
  - image_name: nome dell'immagine.
  - folder: cartella dove salvare le immagini.
  '''

  global dict_people                                                                  # dizionario globale

  image_name = image_name[:3]                                                         # prende i primi 3 caratteri dell'immagine (ID)

  if image_name not in dict_people:                                                   # se non è presente nel dizionario
    dict_people[image_name] = 1                                                       # inizializza a 1
  else:
    dict_people[image_name] += 1                                                      # altrimenti incrementa di 1

  for img in imgs:                                                                    # per ogni immagine
    imageio.imwrite(f'{folder}/{image_name}_{dict_people[image_name]}.jpg', img)      # salva l'immagine nel file
    dict_people[image_name] += 1                                                      # incrementa il contatore

# -- -- # -- -- # -- -- # -- -- # -- -- # -- -- # -- -- #

def BalanceImages(image_folder, output_folder, multiplier, max_morph_samples):
  '''
  Funzione per bilanciare le immagini in una cartella.

  Parametri:
  - image_folder: cartella con le immagini da bilanciare.
  - output_folder: cartella dove salvare le immagini bilanciate.
  - multiplier: numero di immagini generate per ogni immagine originale.
  - max_morph_samples: numero massimo di immagini da morph.
  '''

  if not os.path.exists(output_folder):                                           # crea la directory di output se non esiste
    os.makedirs(output_folder)

  for class_name in os.listdir(image_folder):                                     # itera attraverso tutte le directory (classi) nell'input folder
    class_input_folder = os.path.join(image_folder, class_name)                   # percorso della cartella della classe specifica in input
    class_output_folder = os.path.join(output_folder, class_name)                 # percorso della cartella della classe specifica nell'output folder

    if not os.path.exists(class_output_folder):                                   # crea la directory della classe per le immagini ridimensionate se non esiste
      os.makedirs(class_output_folder)

    if class_name == 'bona_fide':                                                 # se è la classe bona_fide
      for image_name in os.listdir(class_input_folder):                           # itera attraverso ogni immagine nella cartella della classe
        image_path = os.path.join(class_input_folder, image_name)                 # percorso completo dell'immagine
        augmented_imgs = dataAugmentation(image_path, multiplier=multiplier)      # genera le nuove immagini
        saveToFile(augmented_imgs, image_name, class_output_folder)                           # salva le immagini generate nel file
    else:
      imgs_list = os.listdir(class_input_folder)                                  # lista di tutte le immagini nella cartella della classe
      random.shuffle(imgs_list)                                                   # mescola la lista di immagini
      for image_name in imgs_list[:max_morph_samples]:
        image_path = os.path.join(class_input_folder, image_name)                 # percorso completo dell'immagine
        new_image_path = os.path.join(class_output_folder, image_name)            # percorso completo dell'immagine
        ! mv "{image_path}" "{new_image_path}"

# -- -- # -- -- # -- -- # -- -- # -- -- # -- -- # -- -- #

BalanceImages('/content/Resized', '/content/Balanced', multiplier=6, max_morph_samples=1326)

In [ ]:
print("Numero di immagini bona_fide:\t%d" % len(os.listdir('/content/Balanced/bona_fide')))
print("Numero di immagini morphed:\t%d" % len(os.listdir('/content/Balanced/morphed')))

Numero di immagini bona_fide:	1224
Numero di immagini morphed:	1326


<a name="5"></a>
# **Salvataggio**

In [ ]:
#@title Creazione del zip

#@markdown Nome del zip da scaricare:<br>
#@markdown <font color="#ed7d31"><b>Non inlcudere l'estensione (.zip), caratteri speciali o spazi!</b></font>
ZIP_NAME = 'Dataset_Bilanciato_con_ID' #@param {type:"string"}

!mv /content/AMSL /content/AMSL_temp       # cambiamo nome alla cartella AMSL
!mv /content/Balanced /content/AMSL        # rinominiamo la cartella con le immagini pre-processate in AMSL
!zip -r /content/{ZIP_NAME}.zip AMSL       # zippiamo (è importante che la root del zip sia una cartella AMSL)
!mv /content/AMSL /content/Balanced        # rinominiamo la cartella con le immagini pre-processate al nome originale
!mv /content/AMSL_temp /content/AMSL       # rinominiamo la cartella originale in AMSL

In [ ]:
files.download(f'/content/{ZIP_NAME}.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>